# Get summary of crop info for maps

In [ ]:
import os
import sys

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

from mosaic import *
from ras_tools import *
#from var_dataframe import *

## get summary for # rotations per year
or any single variable previously created with 'ts_composite' tool

In [ ]:
#out_dir = '/home/downspout-cel/paraguay_lc/vector/tests'
#out_path = Path('/home/downspout-cel/paraguay_lc/mosaics')
#cell_list = '/home/downspout-cel/paraguay_lc/mosaics/lists/Training_cells.csv'
#ts_dir = '/home/downspout-cel/paraguay_lc/stac/grids'
#ptfile = '/home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Mar2024_ALL.csv'

#numrot_wet_mosaic = mosaic_cells(cell_list, ts_dir, 'comp/kndvi', 'numrot_wet_2021', out_path)
#numrot_wet_mosaic = '/home/downspout-cel/paraguay_lc/mosaics/numrot_wet_2021_mosaic.tif'
#numrot_yr_mosaic = mosaic_cells(cell_list, ts_dir, 'comp/kndvi', 'numrot_yr_2021', out_path)
#numrot_yr_mosaic = '/home/downspout-cel/paraguay_lc/mosaics/numrot_yr_2021_mosaic.tif'

#samp_numrot_wet = get_variables_at_pts_external2(out_dir, numrot_wet_mosaic, ptfile)

#samp_numrot_yr = pd.read_csv(os.path.join(out_dir,'samp_numrot_yr_2021.csv'))
#samp_numrot_wet = pd.read_csv(os.path.join(out_dir,'samp_numrot_wet_2021.csv'))
#samp_numrot_dry = pd.read_csv(os.path.join(out_dir,'samp_numrot_dry_2021.csv'))
                              
#crop_samp_wet = samp_numrot_wet[(samp_numrot_wet['LC_UNQ'] > 30) & (samp_numrot_wet['LC_UNQ'] < 50)]
#crop_samp_yr = samp_numrot_yr[(samp_numrot_yr['LC_UNQ'] > 30) & (samp_numrot_yr['LC_UNQ'] < 50)]
#crop_samp_wetg = crop_samp_wet[['Class','numrot_wet']].groupby('Class').mean()
#crop_samp_yrg = crop_samp_yr[['Class','numrot_yr']].groupby('Class').mean()
#crop_samp_numrot = crop_samp_yrg.join(crop_samp_wetg,on='Class',how='left')
#print(crop_samp_numrot)

## get map classification for sample

In [ ]:
out_dir = '/home/downspout-cel/paraguay_lc/lc_prods'
ptfile = '/home/downspout-cel/paraguay_lc/vector/pts_calval/EPy_district_samp/GENERAL_AUXSAMP.csv'
product_name = "ESA_WorldCereal"
out_name = f'smsamp_2021_{product_name}.csv'
ras = '/home/downspout-cel/paraguay_lc/lc_prods/ESA_WorldCereal/WorldCereal_TempCrops_PyE.tif'
#ras = '/home/downspout-cel/paraguay_lc/mosaics/CEL_base4NoPoly_bal300mix3_mosaic.tif'

samp = get_variables_at_pts_external(out_dir, ras, ptfile,'map_val',out_name)

In [ ]:
samp.head()

## get crop/no_crop confusion matrix for map

In [ ]:
lut_valid = ('../Class_LUT.csv')
lut_map = '/home/downspout-cel/paraguay_lc/lc_prods/ESA_WorldCereal/LUT_ESA_WorldCereal.csv'

#samp = pd.read_csv(os.path.join(out_dir,'smsamp_2021_ESAWorldCereal.csv'))
cm_cropNoCrop = get_confusion_matrix_generic(samp,'map_val','LC_UNQ',lut_valid,lut_map,'LC2',nodata=None, 
                                     print_cm=False, out_dir=out_dir,model_name=product_name)
cm_cropNoCrop

In [ ]:
fixed_ho_dir = '/home/downspout-cel/paraguay_lc/vector/pts_calval/EPy_district_samp'

test_df_all = os.path.join(fixed_ho_dir, 'GENERAL_AUXSAMP.csv')
if os.path.isfile(test_df_all):
    test_all = pd.read_csv(test_df_all)
    ho_smallCrop_path = test_all.loc[(test_all['LC_UNQ'] == 35) | (test_all['LC_UNQ'] == 23)] 
    ho_bigCrop_path = test_all.loc[(test_all['LC2'] == 30) & (test_all['LC_UNQ'] != 35) & (test_all['LC_UNQ'] != 23) & (test_all['LC_UNQ'] < 40)]
    ho_noCrop_path = test_all.loc[(test_all['LC2'] == 0) & (test_all['LC_UNQ'] != 19)]
else:
    sys.stderr.write(f'ERR: cannot find fixed test set')

score = {}
ho_smallcrop = get_variables_at_pts_external(out_dir, ras, ho_smallCrop_path,'map_val',out_name)[['map_val','LC2','OID_','LC_UNQ']]
num_sm_correct = ho_smallcrop['map_val'].sum()
ho_bigcrop = get_variables_at_pts_external(out_dir, ras, ho_bigCrop_path,'map_val',out_name)[['map_val','LC2','OID_','LC_UNQ']]
#ho_bigcrop.to_csv(os.path.join(out_dir, 'crop_test.csv'))
num_big_correct = ho_bigcrop['map_val'].sum()
ho_nocrop = get_variables_at_pts_external(out_dir, ras, ho_noCrop_path,'map_val',out_name)[['map_val','LC2','OID_','LC_UNQ']]
num_no_correct = len(ho_nocrop) - (ho_nocrop['map_val']).sum()
ho = pd.concat([ho_smallcrop,ho_bigcrop,ho_nocrop])
score["recall_smallCrop"] = (num_sm_correct / len(ho_smallcrop)).round(3)
score["recall_bigCrop"] = (num_big_correct / len(ho_bigcrop)).round(3)
score["recall_noCrop"] =  (num_no_correct / len(ho_nocrop)).round(3)
cm = get_confusion_matrix_generic(ho,'map_val','LC_UNQ',lut_valid,lut_map,'LC2',nodata=None, 
        print_cm=False, out_dir=out_dir,model_name=product_name)
score["Kappa_cnc"] = cm.at['crop','Kappa']
score["F1_cnc"] = cm.at['crop','F1']
score["F_5_cnc"] = cm.at['crop','F_5']
score["F_25_cnc"] = cm.at['crop','F_25']        
score["OA_cnc"] = cm.at['All','UA']

print(score)